In [12]:
import sympy as sp
import numpy as np
from matplotlib import pyplot as plt
import sympy.stats
import numpy as np
%run Fitcode.ipynb
alpha, lambd, delta_m, delta, n = sp.symbols('alpha lambda delta_m delta n')
print('initialised')
def foutpropagatie(expr, parameters, sigmas):
    sigmakwadr = 0
    for indx in range(len(parameters)):
        param = sp.diff(expr, parameters[indx])
        sigm = sigmas[indx]
        sigmakwadr += (param*sigm)**2
    #display(sigmakwadr)
    return sigmakwadr

def data_analyse(expr, parameters, meetresultaten): #meetresultaten is een matrix [[param1, sigma1,'N'/'U'], [param2, sigma2,'N'/'U'],...]
    waarde = expr.copy()
    afwijking = 0
    fouten = []
    for resultaat in meetresultaten:
        if resultaat[2] == 'U':
            sigma = sp.sqrt((resultaat[1])**2/12)
        else:
            sigma = resultaat[1]
        fouten.append(sigma)
    sigmakwadr = foutpropagatie(expr, parameters, fouten)
    
    for indx in range(len(parameters)):
        param = parameters[indx]
        result = meetresultaten[indx][0]
        waarde = sp.N(waarde.subs(param, result))
        sigmakwadr = sp.N(sigmakwadr.subs(param, result))
    return [waarde, sp.sqrt(sigmakwadr)]

def multiple_analysis(expr, parameters, metingen): #evt. met multithreading op te lossen
    """
    metingen is een matrix [ [[param1_exp1, sigma1_exp1,'N'/'U'], [param2_exp1, sigma2_exp1,'N'/'U'],...],
                             [[param1_exp2, sigma1_exp2,'N'/'U'], [param2_exp2, sigma2_exp2,'N'/'U'],...],
                             ...
    
    ]
    """
    data = []
    for meting in metingen:
        data.append(data_analyse(expr,parameters,meting))
    dat = np.array(data)
    return dat

def gemiddelde(waarden): #bepaalt het gemiddelde van N getallen en de fout op het gemiddelde
                         #kan ook werken met een array van arrays van getallen (meerdere berekeningen tegelijk)
    if type(waarden) != np.array:
        waarden = np.array(waarden)
    dimensies = waarden.ndim
    if dimensies == 1:
        som = np.sum(waarden)
        N = len(waarden)
        avg = som/N
    elif dimensies == 2:
        som = np.array([np.sum(waarden[i]) for i in range(len(waarden))])
        N = len(waarden[0])
        avg = som/N
    sigmasqsom = 0
    waardentranspose = waarden.T
    for elem in waardentranspose:
        sigmasqsom += (elem-avg)**2
    sigmasqsom /= N*(N-1)
    sigma = np.sqrt(sigmasqsom)
    terug = 'eej foemp fix uw dimensies'
    if dimensies == 1:
        terug = [avg, sigma, 'S']
    elif dimensies == 2:
        terug = [[avg[i],sigma[i],'S'].copy() for i in range(len(waarden))]
    return terug
    

def mu_sigma(waarden): #waarden met hun fout; bepaalt het gemiddelde en de meetfout (statistisch en/of meetfout)
    fout = waarden[0][1]
    gewogengemiddelde = False
    for waarde in waarden:
        if waarde[1] != fout:
            gewogengemiddelde = True
    if not gewogengemiddelde:
        #gemiddelde
        value = 0
        n = len(waarden)
        for waarde in waarden:
            value += waarde[0]
        mu = value/n
        #standaardafwijking
        value = 0
        for waarde in waarden:
            value += (mu-waarde[0])**2
        value /= (n-1)
        sigma_stat = np.sqrt(value)
    else:
        value = 0
        gewicht = 0
        n = len(waarden)
        for waarde in waarden:
            gew = 1/(waarde[1]**2)
            value += gew*waarde[0]
            gewicht += gew
        mu = value/gewicht
        sigma = 1/sp.sqrt(gewicht)
        return (mu, sigma, 'fout bij niet-constante meetfout')
    #welke fout teruggeven?
    if sigma_stat/fout > 10:
        print('statistische fout')
        return (mu, sigma_stat, 'N')
    elif sigma_stat == 0:
        print('fout van het meettoestel')
        return (mu, fout, 'U')
    elif fout/sigma_stat > 10:
        print('fout van het meettoestel')
        return (mu, fout, 'U')
    else:
        return (mu, sigma_stat, fout)
    
def normaaltest(hypothese, gemeten): #gemeten is [mu, sigma] merk op dat sigma de steekproefstand.afwijking is!
    Zscore = (hypothese-gemeten[0])/gemeten[1]
    Z = sympy.stats.Normal('Z', 0,1)
    if Zscore >0:
        return 2*sympy.stats.P(Z > Zscore)
    else:
        return 2*sympy.stats.P(Z < Zscore)

def Ttest(hypothese, gemeten, vrijheidsgraden):
    Zscore = (hypothese-gemeten[0])/gemeten[1]
    Z = sympy.stats.StudentT('Z', vrijheidsgraden)
    if Zscore >0:
        return 2*sympy.stats.P(Z > Zscore)
    else:
        return 2*sympy.stats.P(Z < Zscore)    

def test_sigma1issigma2(sigma1,sigma2, n1, n2, p_waarde = 0.05, return_p = False):
    sigmamax = max(sigma1, sigma2)
    sigmamin = min(sigma1, sigma2)
    Fvalue = sigmamax**2/sigmamin**2 #altijd >= 1
    F = sympy.stats.FDistribution('F',n1-1,n2-1)
    kans = 2*sympy.stats.P(F > Fvalue)
    kans = kans.evalf()
    if return_p:
        return kans
    else:
        if kans >= p_waarde:
            return True #de twee sigma's zijn gelijk
        else:
            return False
    
def test_mu1ismu2(meting1, meting2, n1, n2, p_waarde = 0.05, return_p = False, cutoff_normaal = 30):
    mu1, sigma1, _ = meting1
    mu2, sigma2, _ = meting2
    sigma1_is_sigma2 = test_sigma1issigma2(sigma1, sigma2, n1, n2, p_waarde)
    if sigma1_is_sigma2:
        print('standaardafwijkingen zijn gelijk')
        SIGMA = np.sqrt(((n1-1)*sigma1**2 +(n2-1)*sigma2**2)/ (n1+n2-2))
        noemer = SIGMA*np.sqrt(1/n1 + 1/n2)
        Twaarde = abs((mu1-mu2)/noemer)
        print('T = ', end = ' ')
        print(Twaarde)
        if n1 + n2 -2 < cutoff_normaal:
            T = sympy.stats.StudentT('T',n1+n2-2)
        else: #dat ding kan niet rekenen met te veel vrijheidsgraden in de T-test
            T = sympy.stats.Normal('T',0, 1)
        kans = 2*sympy.stats.P(T > Twaarde)
        print('p =',end = ' ')
        kans = kans.evalf()
        print(kans)

    else:
        print('standaardafwijkingen zijn ongelijk')
        r = (sigma1**2/n1 + sigma2**2/n2)**2 / ( (sigma1**2/n1)**2/(n1-1) + (sigma2**2/n2)**2/(n2-1))
        noemer = np.sqrt(sigma1**2/n1+sigma2**2/n2)
        Twaarde = abs((mu1-mu2)/noemer)
        print('r = ', end = ' ')
        print(r)
        if r < cutoff_normaal:
            r = sp.floor(r)
            T = sympy.stats.StudentT('T',r)
        else: #dat ding kan niet rekenen met te veel vrijheidsgraden in de T-test
            T = sympy.stats.Normal('T',0, 1)
        print('T = ',end = ' ');print(Twaarde)
        kans = 2*sympy.stats.P(T > Twaarde)
        print('p = ',end = ' ')
        kans = kans.evalf()
        print(kans)
        
    
    if return_p:
        return kans
    else:
        if kans < p_waarde:
            return False
        else:
            return True

def latex_print_tabel(meetwaarden, namen):
    """
    namen = lijst [param1naam, param2naam, param3naam, ...], de hoofdingen van de tabel
    meetwaarden = matrix [ [param1meting1, param2meting1, param3meting1,...],
                           [param1meting2, param2meting2, param3meting3,...],
                        ]
                met elke paramimetingj = [waarde, fout] (het standaard formaat)
    """


    print('\\begin{table}[h!]')
    tabular = '\\begin{tabular}{' + '|c'*len(namen) + '|}'
    print(tabular)

    #de hoofdingen
    print('\\hline')
    for naam in namen[:-1]:
        print(naam, end = '&')
    print(namen[-1],end = '\\\\')

    #de meetwaarden
    for reeks in meetwaarden:
        print('\\hline')
        for meting in reeks[:-1]:
            output = latex_print_meting(meting, printing = False)
            print(output, end = '&')
        print(latex_print_meting(reeks[-1], printing = False), end = '\\\\')
    print('\\hline')
    print('\\end{tabular}\\end{table}')

def latex_print_meting(meetwaarde, naam = None, printing = True):
    #meetwaarden is de standaard vector [waarde, fout] met eventueel 'U'/'N' erachter
    waarde, fout = meetwaarde[0],meetwaarde[1]
    exponent = 0
    TUPPEL = False

    if type(fout) == tuple:
        print(meetwaarde) #dit moet ik eens fixen als ik goesting heb

        if 0.9 <= fout[1]/fout[0] <= 1.1:
            fout = max(fout)
        else:
            TUPPEL = True
            TUPPELVAL = fout
            fout = max(fout)

    while not 1 <= fout*(10**(-1*exponent)) < 100:
        print(fout, exponent)
        if 1 < fout:
            exponent += 1
        else:
            exponent -= 1
    waarde /= 10**exponent
    fout /= 10**exponent
    if TUPPEL:
        foutlinks = TUPPELVAL[0]/10**exponent
        foutrechts = TUPPELVAL[1]/10**exponent
    waarde = round(waarde, 1)
    fout = round(fout,1)
    if not TUPPEL:
        latex_output = f'(%s \\pm %s)\\cdot 10^{{%s}}$'%(waarde, fout,exponent)
    else:
        latex_output = f'%s^{{-%s}}_{{+%s}}\\cdot 10^{{%s}}$'%(waarde, foutlinks, foutrechts, exponent)
    if naam is None:
        latex_output = '$' + latex_output
    else:
        latex_output = '$' + naam +' = ' + latex_output
    if printing:
        print(latex_output)
    return latex_output

initialised
